In [ ]:
!pip uninstall -y langchain langchain-community langchain-core langchain-groq
!pip install langchain==0.1.20 langchain-community==0.0.38 langchain-core==0.1.52 langchain-groq==0.1.3 langchain-text-splitters cassio python-dotenv
print("\n✅ Download complete!.")

In [ ]:
import subprocess
import time

print("The Ollama server is preparing...")
!curl -fsSL https://ollama.com/install.sh | sh

print("The server is starting up...")
process = subprocess.Popen("ollama serve", shell=True)
time.sleep(5)

print("Model is downloading (llama3.2:3b)...")
!ollama pull llama3.2:3b

print("\n✅ OLLAMA is Ready!")

In [ ]:
import os
import bs4
import cassio
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Cassandra
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


GROQ_API_KEY = "Your API Key"

ASTRA_DB_TOKEN = "Your Astra DB Token"
ASTRA_DB_ID = "Your Astra DB"

os.environ["USER_AGENT"] = "MyColabApp/1.0"

print("Connecting to the database...")
try:
    cassio.init(token=ASTRA_DB_TOKEN, database_id=ASTRA_DB_ID)
    print("✅ Database Connection Successful!")
except Exception as e:
    print("\n❌ Database Error! Make sure the database is ‘Active’ (Green) in the Astra DB panel.")
    print("Hata Detayı:", e)
    raise e

# EMBEDDING VE VECTOR DATABASE
print("The Ollama model is being prepared...")
embeddings = OllamaEmbeddings(model="llama3.2:3b")

astra_vector_store = Cassandra(
    embedding=embeddings,
    table_name="qa_mini_demo_simple",
    session=None
)

#  DATA RETRIEVAL
print("Web data is being retrieved...")
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(parse_only=bs4.SoupStrainer(
        class_=("post-title", "post-content")
    ))
)
text_documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(text_documents)


# RAG CHAİNS
print("Groq is being prepared...")

llm = ChatGroq(
    groq_api_key=GROQ_API_KEY,
    model_name="llama-3.3-70b-versatile"
)

prompt = ChatPromptTemplate.from_template("""
Answer the question based ONLY on the following context:
{context}

Question: {input}
""")

retriever = astra_vector_store.as_retriever()
document_chain = create_stuff_documents_chain(llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)


print("\n--- SONUÇ ---")
response = retrieval_chain.invoke({"input": "What is Chain of Thought?"})
print(f"\nCEVAP: {response['answer']}")

In [ ]:
print("\n--- RAG CHAT BOT HAS STARTED (Type ‘q’ to exit) ---")

while True:
    user_input = input("\Write your question: ")

    if user_input.lower() == 'q':
        print("See u!")
        break

    # Soruyu zincire gönder
    response = retrieval_chain.invoke({"input": user_input})

    print(f"\n🤖 CEVAP: {response['answer']}")
    print("-" * 30)